In [ ]:
# setup
!pip -q install -U PyYAML
from altair.vega import Vega
Vega.renderers.enable('colab')
%load_ext altair

     |████████████████████████████████| 276kB 2.7MB/s 


In [ ]:
%%vega
{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A radar chart example, showing multiple dimensions in a radial layout.",
  "width": 400,
  "height": 400,
  "padding": 100,
  "autosize": {"type": "none", "contains": "padding"},

  "signals": [
    {"name": "radius", "update": "width / 2"},
  ],

  "data": [
    {
      "name": "table",
      "values": [
        {"key": "Måle parameter", "value": 19, "category": 0},
        {"key": "Måle rækkevide", "value": 22, "category": 0},
        {"key": "Sensitivitet", "value": 14, "category": 0},
        {"key": "Nøjagtighed", "value": 38, "category": 0},
        {"key": "Præcision", "value": 23, "category": 0},
        {"key": "Opløsning", "value": 5, "category": 0},
        {"key": "Sampling hastighed", "value": 27, "category": 0},
        {"key": "Lineæritet", "value": 13, "category": 0},
        {"key": "Usikkerhed", "value": 12, "category": 0},
        {"key": "Respons tid", "value": 42, "category": 0},
        {"key": "Stabiliserings tid", "value": 13, "category": 0},
        {"key": "Påkrævet antal sensor", "value": 6, "category": 0},
        {"key": "Måle parameter", "value": 40, "category": 1},
        {"key": "Måle rækkevide", "value": 40, "category": 1},
        {"key": "Sensitivitet", "value": 20, "category": 1},
        {"key": "Nøjagtighed", "value": 10, "category": 1},
        {"key": "Præcision", "value": 53, "category": 1},
        {"key": "Opløsning", "value": 15, "category": 1},
        {"key": "Sampling hastighed", "value": 10, "category": 1},
        {"key": "Lineæritet", "value": 13, "category": 1},
        {"key": "Usikkerhed", "value": 12, "category": 1},
        {"key": "Respons tid", "value": 42, "category": 1},
        {"key": "Stabiliserings tid", "value": 13, "category": 1},
        {"key": "Påkrævet antal sensor", "value": 6, "category": 1},

      ]
    },
    {
      "name": "keys",
      "source": "table",
      "transform": [
        {
          "type": "aggregate",
          "groupby": ["key"]
        }
      ]
    }
  ],

  "scales": [
    {
      "name": "angular",
      "type": "point",
      "range": {"signal": "[-PI, PI]"},
      "padding": 0.5,
      "domain": {"data": "table", "field": "key"}
    },
    {
      "name": "radial",
      "type": "linear",
      "range": {"signal": "[0, radius]"},
      "zero": True,
      "nice": False,
      "domain": {"data": "table", "field": "value"},
      "domainMin": 0
    },
    {
      "name": "color",
      "type": "ordinal",
      "domain": {"data": "table", "field": "category"},
      "range": {"scheme": "category10"}
    }
  ],

  "encode": {
    "enter": {
      "x": {"signal": "radius"},
      "y": {"signal": "radius"}
    }
  },

  "marks": [
    {
      "type": "group",
      "name": "categories",
      "zindex": 1,
      "from": {
        "facet": {"data": "table", "name": "facet", "groupby": ["category"]}
      },
      "marks": [
        {
          "type": "line",
          "name": "category-line",
          "from": {"data": "facet"},
          "encode": {
            "enter": {
              "interpolate": {"value": "linear-closed"},
              "x": {"signal": "scale('radial', datum.value) * cos(scale('angular', datum.key))"},
              "y": {"signal": "scale('radial', datum.value) * sin(scale('angular', datum.key))"},
              "stroke": {"scale": "color", "field": "category"},
              "strokeWidth": {"value": 1},
              "fill": {"scale": "color", "field": "category"},
              "fillOpacity": {"value": 0.1}
            }
          }
        },
        {
          "type": "text",
          "name": "value-text",
          "from": {"data": "category-line"},
          "encode": {
            "enter": {
              "x": {"signal": "datum.x"},
              "y": {"signal": "datum.y"},
              "text": {"signal": "datum.datum.value"},
              "align": {"value": "center"},
              "baseline": {"value": "middle"},
              "fill": {"value": "black"}
            }
          }
        }
      ]
    },
    {
      "type": "rule",
      "name": "radial-grid",
      "from": {"data": "keys"},
      "zindex": 0,
      "encode": {
        "enter": {
          "x": {"value": 0},
          "y": {"value": 0},
          "x2": {"signal": "radius * cos(scale('angular', datum.key))"},
          "y2": {"signal": "radius * sin(scale('angular', datum.key))"},
          "stroke": {"value": "lightgray"},
          "strokeWidth": {"value": 1}
        }
      }
    },
    {
      "type": "text",
      "name": "key-label",
      "from": {"data": "keys"},
      "zindex": 1,
      "encode": {
        "enter": {
          "x": {"signal": "(radius + 5) * cos(scale('angular', datum.key))"},
          "y": {"signal": "(radius + 5) * sin(scale('angular', datum.key))"},
          "text": {"field": "key"},
          "align": [
            {
              "test": "abs(scale('angular', datum.key)) > PI / 2",
              "value": "right"
            },
            {
              "value": "left"
            }
          ],
          "baseline": [
            {
              "test": "scale('angular', datum.key) > 0", "value": "top"
            },
            {
              "test": "scale('angular', datum.key) == 0", "value": "middle"
            },
            {
              "value": "bottom"
            }
          ],
          "fill": {"value": "black"},
          "fontWeight": {"value": "bold"}
        }
      }
    },
    {
      "type": "line",
      "name": "outer-line",
      "from": {"data": "radial-grid"},
      "encode": {
        "enter": {
          "interpolate": {"value": "linear-closed"},
          "x": {"field": "x2"},
          "y": {"field": "y2"},
          "stroke": {"value": "lightgray"},
          "strokeWidth": {"value": 1}
        }
      }
    }
  ]
}